# Time-Frequency Consistency (TFC) + CNN PFF

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
from datetime import datetime
import lightning as L
import torch
import torchmetrics
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import CSVLogger

from minerva.analysis.metrics.balanced_accuracy import BalancedAccuracy
from minerva.analysis.model_analysis import TSNEAnalysis
from minerva.data.data_modules.har import MultiModalHARSeriesDataModule
from minerva.data.data_modules.har_rodrigues_24 import HARDataModuleCPC
from minerva.models.loaders import FromPretrained
from minerva.models.nets.time_series.cnns import CNN_PF_Backbone
from minerva.models.ssl.tfc import TFC_Model, TFC_Backbone
from minerva.pipelines.lightning_pipeline import SimpleLightningPipeline

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
execution_id = f'run_{datetime.now().strftime("%Y%m%d-%H%M%S")}'
log_dir = f"./logs/{execution_id}" 

print(f"Execution ID: {execution_id}")
print(f"Log dir: {log_dir}")

Execution ID: run_20260203-190025
Log dir: ./logs/run_20260203-190025


## Pre-training with TFC

### 1. Defining the Data Module

In [4]:
data_module = HARDataModuleCPC(
    data_path="/workspaces/HIAAC-KR-Dev-Container/shared_data/rodrigues_2024_datasets/1-1/wisdm",
    input_size=6,
    window=60,
    overlap=60,
    batch_size=64
)

data_module

HARDataModuleCPC(batch_size=64, datasets=/workspaces/HIAAC-KR-Dev-Container/shared_data/rodrigues_2024_datasets/1-1/wisdm)

### 2. Defining the TF-C Model

In [5]:
# Get a batch of data
data_module.setup("fit")
dataset = data_module.train_dataloader()
batch_x, batch_y = next(iter(dataset))

# Create the model and forward the batch to check the output shape
g_enc = CNN_PF_Backbone(include_middle=True)
r = g_enc.forward(batch_x)
r.shape

torch.Size([64, 64, 12])

In [6]:
backbone = TFC_Backbone(
    input_channels=6,
    TS_length=60,
    single_encoding_size=128,
    time_encoder=CNN_PF_Backbone(include_middle=True,flatten=True),
    frequency_encoder=CNN_PF_Backbone(include_middle=True,flatten=True),
)
model = TFC_Model(
    input_channels=6,
    batch_size=64,
    # TS_length=r.shape[-1],
    TS_length=60,
    num_classes=6,
    single_encoding_size=128,
    backbone=backbone,
    pred_head=None,
)

model

[W203 19:00:30.270438180 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W203 19:00:30.271792848 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W203 19:00:30.272081494 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W203 19:00:30.272374632 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W203 19:00:30.279724563 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W203 19:00:30.280193013 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W203 19:00:30.280538293 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W203 19:00:30.280890438 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.


TFC_Model(
  (backbone): TFC_Backbone(
    (time_encoder): CNN_PF_Backbone(
      (first_padder): ZeroPadder2D(pad_at=(3,), padding_size=2)
      (upper_part): Sequential(
        (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
        (1): ReLU()
        (2): MaxPool2d(kernel_size=(2, 3), stride=(2, 3), padding=1, dilation=1, ceil_mode=False)
      )
      (lower_part): Sequential(
        (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
        (1): ReLU()
        (2): MaxPool2d(kernel_size=(2, 3), stride=(2, 3), padding=1, dilation=1, ceil_mode=False)
      )
      (middle_part): Sequential(
        (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
        (1): ReLU()
        (2): MaxPool2d(kernel_size=(2, 3), stride=(2, 3), padding=1, dilation=1, ceil_mode=False)
      )
      (shared_part): Sequential(
        (0): Conv2d(48, 64, kernel_size=(3, 5), stride=(1, 1))
        (1): ReLU()
        (2): MaxPool2d(kernel_size=(2, 3), stride=(2, 3), padding=1, dilation=1, ce

### 3. Defining the Pytorch Lightning Trainer Configuration

In [7]:
## Callbacks
checkpoint_callback = ModelCheckpoint(
    dirpath='checkpoints/',
    monitor='val_loss',
    mode='min',
    save_last=True
)

## Logger
logger = CSVLogger(save_dir=log_dir, name='tfc-cnn-pff-pretraining', version=execution_id)

## Trainer
trainer = L.Trainer(
    max_epochs=3,
    accelerator="gpu",
    devices=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    # Only for testing. Remove for production. We will only train using 1 batch
    limit_train_batches=1,
    limit_val_batches=1,
)

trainer

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1)` was configured so 1 batch per epoch will be used.
`Trainer(limit_val_batches=1)` was configured so 1 batch will be used.


### 4. Creating the training pipeline (and running the training)

In [8]:
train_pipeline = SimpleLightningPipeline(
    model=model,
    trainer=trainer,
    log_dir=log_dir,
    save_run_status=True,
    seed=42
)
train_pipeline.run(data_module, task="fit")

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:658: Checkpoint directory /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/codecarbon/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type            | Params | Mode 
-----------------------------------------------------
0 | backbone | TFC_Backbone    | 553 K  | train
1 | loss_fn  | NTXentLoss_poly | 0      | train
-----------------------------------------------------
553 K     Trainable params
0         Non-trainable params
553 K     Total params
2.215     Total estimated model params size (MB)
55        Modules in train mode
0         Modules in eval mode


** Seed set to: 42 **
Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/codecarbon/logs/run_20260203-190025/run_2026-02-03-19-00-306f93d6bd.yaml


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 2: 100%|██████████| 1/1 [00:01<00:00,  0.91it/s, v_num=0025, val_loss=2.210]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 1/1 [00:01<00:00,  0.73it/s, v_num=0025, val_loss=2.210]
⏱️ fit took 5.61s → saved to /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/codecarbon/logs/run_20260203-190025/timings_fit.csv
Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/codecarbon/logs/run_20260203-190025/run_2026-02-03-19-00-306f93d6bd.yaml


### 5. Inspecting checkpoints

In [9]:
ckpt_path = checkpoint_callback.last_model_path
ckpt = torch.load(ckpt_path, map_location="cuda")
ckpt = ckpt.get("state_dict", ckpt)
list(ckpt.keys())

['backbone.time_encoder.upper_part.0.weight',
 'backbone.time_encoder.upper_part.0.bias',
 'backbone.time_encoder.lower_part.0.weight',
 'backbone.time_encoder.lower_part.0.bias',
 'backbone.time_encoder.middle_part.0.weight',
 'backbone.time_encoder.middle_part.0.bias',
 'backbone.time_encoder.shared_part.0.weight',
 'backbone.time_encoder.shared_part.0.bias',
 'backbone.frequency_encoder.upper_part.0.weight',
 'backbone.frequency_encoder.upper_part.0.bias',
 'backbone.frequency_encoder.lower_part.0.weight',
 'backbone.frequency_encoder.lower_part.0.bias',
 'backbone.frequency_encoder.middle_part.0.weight',
 'backbone.frequency_encoder.middle_part.0.bias',
 'backbone.frequency_encoder.shared_part.0.weight',
 'backbone.frequency_encoder.shared_part.0.bias',
 'backbone.time_projector.projector.0.weight',
 'backbone.time_projector.projector.0.bias',
 'backbone.time_projector.projector.1.module.weight',
 'backbone.time_projector.projector.1.module.bias',
 'backbone.time_projector.projecto

## Fine-tuning with CNN PFF

### 1. Defining the Data Module

In [10]:
data_module = MultiModalHARSeriesDataModule(
    data_path="/workspaces/HIAAC-KR-Dev-Container/shared_data/daghar/standardized_view/MotionSense/",
    feature_prefixes=["accel-x", "accel-y", "accel-z", "gyro-x", "gyro-y", "gyro-z"],
    label="standard activity code",
    features_as_channels=True,
    cast_to="float32",
    batch_size=64,
)

data_module

MultiModalHARSeriesDataModule(data_path=/workspaces/HIAAC-KR-Dev-Container/shared_data/daghar/standardized_view/MotionSense, batch_size=64)

In [11]:
# Pega os dataloaders de treino e validação
data_module.setup("fit")
train_data_loader = data_module.train_dataloader()
validation_data_loader = data_module.val_dataloader()
first_batch = next(iter(train_data_loader))

X, y = first_batch
print(X.shape, y.shape)

Using DataLoader with shuffle=True
Using DataLoader with shuffle=False
torch.Size([64, 6, 60]) torch.Size([64])


### 2. Defining the CNN PFF Model for Fine-tuning

In [12]:
backbone = TFC_Backbone(
    input_channels=6,
    TS_length=60,
    single_encoding_size=128,
    time_encoder=CNN_PF_Backbone(include_middle=True,flatten=True),
    frequency_encoder=CNN_PF_Backbone(include_middle=True,flatten=True),
)


backbone = FromPretrained(
    model=backbone, 
    ckpt_path=checkpoint_callback.best_model_path,
    filter_keys=["backbone"],
    keys_to_rename={"backbone.": ""},      # Let's remove the prefix from the keys
                                        # on the checkpoint to load the model
                                        # correctly
    strict=True,
    error_on_missing_keys=True
)

Performing key renaming with: {'backbone.': ''}
	Renaming key: backbone.time_encoder.upper_part.0.weight -> time_encoder.upper_part.0.weight (changed: True)
	Renaming key: backbone.time_encoder.upper_part.0.bias -> time_encoder.upper_part.0.bias (changed: True)
	Renaming key: backbone.time_encoder.lower_part.0.weight -> time_encoder.lower_part.0.weight (changed: True)
	Renaming key: backbone.time_encoder.lower_part.0.bias -> time_encoder.lower_part.0.bias (changed: True)
	Renaming key: backbone.time_encoder.middle_part.0.weight -> time_encoder.middle_part.0.weight (changed: True)
	Renaming key: backbone.time_encoder.middle_part.0.bias -> time_encoder.middle_part.0.bias (changed: True)
	Renaming key: backbone.time_encoder.shared_part.0.weight -> time_encoder.shared_part.0.weight (changed: True)
	Renaming key: backbone.time_encoder.shared_part.0.bias -> time_encoder.shared_part.0.bias (changed: True)
	Renaming key: backbone.frequency_encoder.upper_part.0.weight -> frequency_encoder.upper

[W203 19:00:39.973854046 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W203 19:00:39.974728253 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W203 19:00:39.975014535 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W203 19:00:39.975312635 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W203 19:00:39.979955423 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W203 19:00:39.980317177 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W203 19:00:39.980575355 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W203 19:00:39.980828804 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.


In [13]:
from minerva.models.nets.mlp import MLP
head = MLP([256, 128, 6])
head

MLP(
  (0): Linear(in_features=256, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=6, bias=True)
)

In [14]:
from minerva.models.nets.base import SimpleSupervisedModel
model = SimpleSupervisedModel(
    backbone=backbone,
    fc=head,
    loss_fn=torch.nn.CrossEntropyLoss(),
    flatten=False,
    train_metrics = {"acc": torchmetrics.Accuracy(task="multiclass", num_classes=6)},
    val_metrics = {"acc": torchmetrics.Accuracy(task="multiclass", num_classes=6)},
    test_metrics = {"acc": torchmetrics.Accuracy(task="multiclass", num_classes=6)}
)

model

SimpleSupervisedModel(
  (backbone): TFC_Backbone(
    (time_encoder): CNN_PF_Backbone(
      (first_padder): ZeroPadder2D(pad_at=(3,), padding_size=2)
      (upper_part): Sequential(
        (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
        (1): ReLU()
        (2): MaxPool2d(kernel_size=(2, 3), stride=(2, 3), padding=1, dilation=1, ceil_mode=False)
      )
      (lower_part): Sequential(
        (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
        (1): ReLU()
        (2): MaxPool2d(kernel_size=(2, 3), stride=(2, 3), padding=1, dilation=1, ceil_mode=False)
      )
      (middle_part): Sequential(
        (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
        (1): ReLU()
        (2): MaxPool2d(kernel_size=(2, 3), stride=(2, 3), padding=1, dilation=1, ceil_mode=False)
      )
      (shared_part): Sequential(
        (0): Conv2d(48, 64, kernel_size=(3, 5), stride=(1, 1))
        (1): ReLU()
        (2): MaxPool2d(kernel_size=(2, 3), stride=(2, 3), padding=1, di

In [15]:
# Option 1: Measure ALL model parameters (including backbone AND head)
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total model parameters: {total_params:,}")
print(f"Trainable model parameters: {trainable_params:,}")

# Option 2: Measure components separately (as you intended)
# Backbone parameters
backbone_total = sum(p.numel() for p in model.backbone.parameters())
backbone_trainable = sum(p.numel() for p in model.backbone.parameters() if p.requires_grad)

# Head parameters (use model.fc, not head variable)
head_total = sum(p.numel() for p in model.fc.parameters())
head_trainable = sum(p.numel() for p in model.fc.parameters() if p.requires_grad)

print(f"\nBackbone:")
print(f"  Total parameters: {backbone_total:,}")
print(f"  Trainable parameters: {backbone_trainable:,}")

print(f"\nClassification Head:")
print(f"  Total parameters: {head_total:,}")
print(f"  Trainable parameters: {head_trainable:,}")

# Verify the sum matches Option 1
print(f"\nVerification:")
print(f"Backbone + Head total: {backbone_total + head_total:,}")
print(f"Should match total: {total_params:,}")

Total model parameters: 587,462
Trainable model parameters: 587,462

Backbone:
  Total parameters: 553,792
  Trainable parameters: 553,792

Classification Head:
  Total parameters: 33,670
  Trainable parameters: 33,670

Verification:
Backbone + Head total: 587,462
Should match total: 587,462


In [16]:
evaluation_data = torch.rand(1000, 6, 60, device='cuda')
model.to('cuda')

SimpleSupervisedModel(
  (backbone): TFC_Backbone(
    (time_encoder): CNN_PF_Backbone(
      (first_padder): ZeroPadder2D(pad_at=(3,), padding_size=2)
      (upper_part): Sequential(
        (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
        (1): ReLU()
        (2): MaxPool2d(kernel_size=(2, 3), stride=(2, 3), padding=1, dilation=1, ceil_mode=False)
      )
      (lower_part): Sequential(
        (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
        (1): ReLU()
        (2): MaxPool2d(kernel_size=(2, 3), stride=(2, 3), padding=1, dilation=1, ceil_mode=False)
      )
      (middle_part): Sequential(
        (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
        (1): ReLU()
        (2): MaxPool2d(kernel_size=(2, 3), stride=(2, 3), padding=1, dilation=1, ceil_mode=False)
      )
      (shared_part): Sequential(
        (0): Conv2d(48, 64, kernel_size=(3, 5), stride=(1, 1))
        (1): ReLU()
        (2): MaxPool2d(kernel_size=(2, 3), stride=(2, 3), padding=1, di

In [17]:
from thop import profile

macs, params = profile(model, inputs=(evaluation_data,))

print(f"MACs: {macs:,}")
print(f"Parameters: {params:,}")

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm1d'>.


MACs: 3,643,904,000.0
Parameters: 587,462.0


In [21]:
evaluation_data = torch.rand(1, 6, 60, device='cuda')
# model.to('cuda')
from thop import profile

macs, params = profile(model, inputs=(evaluation_data,))

print(f"MACs: {macs:,}")
print(f"Parameters: {params:,}")

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm1d'>.
MACs: 3,643,904.0
Parameters: 587,462.0


In [18]:
from thop import profile

macs, params, layer_infos = profile(
    model,
    inputs=(evaluation_data,),
    ret_layer_info=True,
    verbose=False
)

for name, info in layer_infos.items():
    print(info)
    # print(
    #     f"{name:40s} | "
    #     f"MACs: {info['macs']:,} | "
    #     f"Params: {info['params']:,}"
    # )


(3610368000.0, 553792.0, {'time_encoder': (1574784000.0, 46624.0, {'first_padder': (0.0, 0, {}), 'upper_part': (25056000.0, 160.0, {'0': (25056000.0, 160.0, {}), '1': (0.0, 0.0, {}), '2': (0.0, 0.0, {})}), 'lower_part': (25056000.0, 160.0, {'0': (25056000.0, 160.0, {}), '1': (0.0, 0.0, {}), '2': (0.0, 0.0, {})}), 'middle_part': (50112000.0, 160.0, {'0': (50112000.0, 160.0, {}), '1': (0.0, 0.0, {}), '2': (0.0, 0.0, {})}), 'shared_part': (1474560000.0, 46144.0, {'0': (1474560000.0, 46144.0, {}), '1': (0.0, 0.0, {}), '2': (0.0, 0.0, {})})}), 'frequency_encoder': (1574784000.0, 46624.0, {'first_padder': (0.0, 0, {}), 'upper_part': (25056000.0, 160.0, {'0': (25056000.0, 160.0, {}), '1': (0.0, 0.0, {}), '2': (0.0, 0.0, {})}), 'lower_part': (25056000.0, 160.0, {'0': (25056000.0, 160.0, {}), '1': (0.0, 0.0, {}), '2': (0.0, 0.0, {})}), 'middle_part': (50112000.0, 160.0, {'0': (50112000.0, 160.0, {}), '1': (0.0, 0.0, {}), '2': (0.0, 0.0, {})}), 'shared_part': (1474560000.0, 46144.0, {'0': (14745

manual

In [19]:
layer_stats = {}
# evaluation_data = torch.rand(1000, 6, 60, device='cuda')
def hook_fn(module, inp, out):
    name = module.__class__.__name__
    params = sum(p.numel() for p in module.parameters())
    layer_stats.setdefault(name, {"params": 0})
    layer_stats[name]["params"] += params

hooks = []
for m in model.modules():
    hooks.append(m.register_forward_hook(hook_fn))

_ = model(evaluation_data)

for h in hooks:
    h.remove()

print(layer_stats)


{'ZeroPadder2D': {'params': 0}, 'Conv2d': {'params': 93248}, 'ReLU': {'params': 0}, 'MaxPool2d': {'params': 0}, 'Sequential': {'params': 553792}, 'CNN_PF_Backbone': {'params': 93248}, 'Linear': {'params': 493190}, 'BatchNorm1d': {'params': 1024}, 'IgnoreWhenBatch1': {'params': 1024}, 'TFC_Standard_Projector': {'params': 460544}, 'TFC_Backbone': {'params': 553792}, 'MLP': {'params': 33670}, 'SimpleSupervisedModel': {'params': 587462}}


In [20]:
a

NameError: name 'a' is not defined

In [ ]:
from codecarbon import EmissionsTracker

torch.cuda.is_available()
torch.cuda.device_count()

1

In [ ]:
import pandas as pd
from codecarbon import EmissionsTracker

TOTAL_RUNS = 14
DISCARD_FIRST = 2
DISCARD_LAST = 2

results = []

tracker = EmissionsTracker(
    project_name="basic_measurement",
    measure_power_secs=10,
    save_to_file=False
)

try:
    for run_id in range(TOTAL_RUNS):
        tracker.start_task(f"measure_inference_{run_id}")

        _ = model(evaluation_data)  # inference

        emissions = tracker.stop_task()

        energy_kwh = emissions.energy_consumed
        energy_mwh = energy_kwh * 1_000          # kWh → mWh
        energy_j   = energy_kwh * 3_600_000      # kWh → J
        emissions_g = emissions.emissions * 1_000 * 1_000  # kg → g→ mg

        results.append({
            "run": run_id,
            "energy_mWh": energy_mwh,
            "energy_J": energy_j,
            "emissions_mgCO2eq": emissions_g,
            # "duration_s": emissions.duration
        })

finally:
    tracker.stop()

# --- All runs ---
df_all = pd.DataFrame(results)

# --- Valid runs (ignore first & last 2) ---
df_valid = df_all.iloc[DISCARD_FIRST: TOTAL_RUNS - DISCARD_LAST]

# --- Statistics ---
mean = df_valid[["energy_mWh", "energy_J", "emissions_mgCO2eq"]].mean()
std  = df_valid[["energy_mWh", "energy_J", "emissions_mgCO2eq"]].std()

summary = pd.DataFrame(
    [mean, std],
    index=["mean", "std"]
).round(2)

# --- Final table ---
df_final = pd.concat([
    df_valid.set_index("run").round(2),
    summary
])

df_final


[codecarbon WARNING @ 02:19:50] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 02:19:50] [setup] RAM Tracking...
[codecarbon INFO @ 02:19:50] [setup] CPU Tracking...
[codecarbon WARNING @ 02:19:51] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist, and are readable, at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 02:19:51] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU E5-2630 v2 @ 2.60GHz
[codecarbon WARNING @ 02:19:51] No CPU tracking mode found. Falling back on CPU load mode.
[codecarbon INFO @ 02:19:51] [setup] GPU Tracking...
[codecarbon INFO @ 02:19:51] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 02:19:51] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: cpu_load
                GPU Tracking Method: pynvml
         

,energy_mWh,energy_J,emissions_mgCO2eq
2,0.01,45.18,1.23
3,0.01,45.27,1.24
4,0.01,45.11,1.23
5,0.01,39.82,1.09
6,0.01,40.24,1.10
7,0.01,45.47,1.24
8,0.01,45.98,1.26
9,0.01,44.96,1.23
10,0.01,40.43,1.10
11,0.01,45.34,1.24


In [ ]:
a

NameError: name 'a' is not defined

### 3. Defining the Pytorch Lightning Trainer Configuration

In [ ]:
## Callbacks
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/", monitor="val_loss", mode="min", save_last=True
)

## Logger
logger = CSVLogger(
    save_dir=log_dir, name="tfc-cnn-pff-finetune", version=execution_id
)

## Trainer
trainer = L.Trainer(
    max_epochs=3,
    accelerator="gpu",
    devices=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    # Only for testing. Remove for production. We will only train using 1 batch
    limit_train_batches=1,
    limit_val_batches=1,
)

trainer

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1)` was configured so 1 batch per epoch will be used.
`Trainer(limit_val_batches=1)` was configured so 1 batch will be used.


### 4. Creating the fine-tuning pipeline (and running the training)

In [ ]:
train_pipeline = SimpleLightningPipeline(
    model=model,
    trainer=trainer,
    log_dir=log_dir,
    save_run_status=True,
    seed=42
)
train_pipeline.run(data_module, task="fit")

Seed set to 42





 cuda device: 


 NVIDIA RTX A5000
Log directory set to: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/logs/run_20250218-190101
Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/logs/run_20250218-190101/run_2025-02-18-19-02-1401fdfefffd4744888e3d3786c55473e2.yaml


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:652: Checkpoint directory /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [3]

  | Name     | Type             | Params | Mode 
------------------------------------------------------
0 | backbone | TFC_Backbone     | 553 K  | train
1 | fc       | MLP              | 33.7 K | train
2 | loss_fn  | CrossEntropyLoss | 0      | train
------------------------------------------------------
587 K     Trainable params
0         Non-trainable params
587 K     Total params
2.350     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]Using DataLoader with shuffle=False
Using DataLoader with shuffle=True                                         


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 2: 100%|██████████| 1/1 [00:08<00:00,  0.12it/s, v_num=0101, val_loss=2.730, val_acc=0.000, train_loss=1.410, train_acc=0.422]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 1/1 [00:08<00:00,  0.12it/s, v_num=0101, val_loss=2.730, val_acc=0.000, train_loss=1.410, train_acc=0.422]
Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/logs/run_20250218-190101/run_2025-02-18-19-02-1401fdfefffd4744888e3d3786c55473e2.yaml


## Evaluating the Fine-tuned Model

In [ ]:
test_pipeline = SimpleLightningPipeline(
    model=model,
    trainer=trainer,
    log_dir=log_dir,
    save_run_status=True,
    seed=42,
    classification_metrics={
        "accuracy": torchmetrics.Accuracy(num_classes=6, task="multiclass"),
        "f1": torchmetrics.F1Score(num_classes=6, task="multiclass"),
        "precision": torchmetrics.Precision(num_classes=6, task="multiclass"),
        "recall": torchmetrics.Recall(num_classes=6, task="multiclass"),
        "balanced_accuracy": BalancedAccuracy(num_classes=6, task="multiclass"),
    },
    apply_metrics_per_sample=False,
    # model_analysis={
    #     "tsne": TSNEAnalysis(
    #         height=800,
    #         width=800,
    #         legend_title="Activity",
    #         title="t-SNE of TFC Finetuned on MotionSense",
    #         output_filename="tsne_tfc_finetuned_motionsense.pdf",
    #         label_names={
    #             0: "sit",
    #             1: "stand",
    #             2: "walk",
    #             3: "stair up",
    #             4: "stair down",
    #             5: "run",
    #             6: "stair up and down",
    #         },
    #     )
    # },
)

test_pipeline.run(
    data_module, task="evaluate", ckpt_path=checkpoint_callback.best_model_path
)

Seed set to 42





 cuda device: 


 NVIDIA RTX A5000
Log directory set to: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/logs/run_20250218-190101
Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/logs/run_20250218-190101/run_2025-02-18-19-02-42b7807a4e71cb4e9fb004c1ebe8346112.yaml
Using DataLoader with shuffle=False


Restoring states from the checkpoint path at /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/checkpoints/epoch=0-step=1-v3.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [3]
Loaded model weights from the checkpoint at /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/checkpoints/epoch=0-step=1-v3.ckpt


Using DataLoader with shuffle=False
Predicting DataLoader 0: 100%|██████████| 17/17 [00:00<00:00, 103.75it/s]
Running classification metrics...
Metrics saved to /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/logs/run_20250218-190101/metrics_2025-02-18-19-02-42b7807a4e71cb4e9fb004c1ebe8346112.yaml
Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/logs/run_20250218-190101/run_2025-02-18-19-02-42b7807a4e71cb4e9fb004c1ebe8346112.yaml


/workspaces/HIAAC-KR-Dev-Container/Minerva-Dev/minerva/analysis/metrics/balanced_accuracy.py:59: UserWarning: y_pred contains nan values and not all classes passed
  warnings.warn(f"y_pred contains nan values and not all classes passed")


{'classification': {'accuracy': [0.32862523198127747],
  'f1': [0.32862523198127747],
  'precision': [0.32862523198127747],
  'recall': [0.32862523198127747],
  'balanced_accuracy': [0.24832433462142944]}}